In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from k8sclient.keywords import switch_cluster, register_cluster
register_cluster("yancheng", "~/.kube/config-yancheng")
register_cluster("shanghai", "~/.kube/config-shanghai")
switch_cluster("yancheng")

In [3]:
from k8sclient.dataframes import list_persistent_volume
pv = list_persistent_volume()

In [5]:
null = None
# print pv[0]
keeptrue_pvc=[]
for p in pv:
    if not p.metadata.annotations:
        continue
    name = p.metadata.name
    if "io.enndata.kubelet/alpha-pvchostpathnode" in p.metadata.annotations:
        if 'io.enndata.user/alpha-pvhostpathmountpolicy' not in p.metadata.annotations or \
        'io.enndata.user/alpha-pvhostpathquotaforonepod' not in p.metadata.annotations:
            continue
        hostpathpvtype = p.metadata.annotations['io.enndata.user/alpha-pvhostpathmountpolicy'] + \
        p.metadata.annotations['io.enndata.user/alpha-pvhostpathquotaforonepod']
        if hostpathpvtype != "keeptrue":
            continue
        if not p.spec.claim_ref:
            continue
        pvc_name = p.spec.claim_ref.name
        pvc_namespace = p.spec.claim_ref.namespace
        hostpathpv = eval(p.metadata.annotations["io.enndata.kubelet/alpha-pvchostpathnode"])
        for h in hostpathpv:
            node = h['NodeName']
            if len(h['MountInfos']) == 1:
                continue
            for m in h["MountInfos"]:
                if m['PodInfo']:
                    continue
                print name, m['HostPath']
        keeptrue_pvc.append((pvc_name, pvc_namespace))
                

In [7]:
from k8sclient.keywords import list_namespaced_deployment

In [5]:
from k8sclient.keywords import list_namespaces
namespaces = list_namespaces()

In [8]:
# print dp[0]
import math
r = []
for namespace in namespaces:
    dp = list_namespaced_deployment(namespace)
    for d in dp:
        pvc_keeptrue = False
        if not d.spec.template.spec.volumes:
            continue
        for v in d.spec.template.spec.volumes:
            if v.persistent_volume_claim:
                pvc = v.persistent_volume_claim.claim_name
                if (pvc, namespace) in keeptrue_pvc:
                    pvc_keeptrue = True
                break
        if not pvc_keeptrue:
            pass
        max_surge = math.ceil(d.spec.replicas*1.25)
        if d.spec.strategy.rolling_update:
            max_surge = int(d.spec.strategy.rolling_update.max_surge)
        max_unavailable = math.ceil(d.spec.replicas*1.25)
        if d.spec.strategy.rolling_update:
            max_unavailable = int(d.spec.strategy.rolling_update.max_unavailable)
        r.append({
            'name': d.metadata.name,
            'namespace': d.metadata.namespace,
            'replicas': int(d.spec.replicas),
            'max_surge': max_surge,
            'max_unavailable': max_unavailable,
            'pvc_keeptrue': pvc_keeptrue
        })
df = pd.DataFrame(r)

In [9]:
df[(df.max_surge > 0) & (df.pvc_keeptrue == True)]

,max_surge,max_unavailable,name,namespace,pvc_keeptrue,replicas
70,1.0,1.0,jupyter,cust-pro,True,1
83,1.0,1.0,pre1-datanode,dataflow-demo-ye,True,3
84,1.0,1.0,pre1-journalnode1,dataflow-demo-ye,True,1
85,1.0,1.0,pre1-journalnode2,dataflow-demo-ye,True,1
86,1.0,1.0,pre1-journalnode3,dataflow-demo-ye,True,1
87,1.0,1.0,pre1-journalnode4,dataflow-demo-ye,True,1
88,1.0,1.0,pre1-journalnode5,dataflow-demo-ye,True,1
89,1.0,1.0,pre1-journalnode6,dataflow-demo-ye,True,1
90,1.0,1.0,pre1-journalnode7,dataflow-demo-ye,True,1
94,1.0,1.0,pre1-namenode1,dataflow-demo-ye,True,1


In [20]:
df[(df.max_surge > 0) & (df.pvc_keeptrue == True)]

,max_surge,max_unavailable,name,namespace,pvc_keeptrue,replicas
70,1.0,1.0,jupyter,cust-pro,True,1
83,1.0,1.0,pre1-datanode,dataflow-demo-ye,True,3
84,1.0,1.0,pre1-journalnode1,dataflow-demo-ye,True,1
85,1.0,1.0,pre1-journalnode2,dataflow-demo-ye,True,1
86,1.0,1.0,pre1-journalnode3,dataflow-demo-ye,True,1
87,1.0,1.0,pre1-journalnode4,dataflow-demo-ye,True,1
88,1.0,1.0,pre1-journalnode5,dataflow-demo-ye,True,1
89,1.0,1.0,pre1-journalnode6,dataflow-demo-ye,True,1
90,1.0,1.0,pre1-journalnode7,dataflow-demo-ye,True,1
94,1.0,1.0,pre1-namenode1,dataflow-demo-ye,True,1


In [21]:
from k8sclient.dataframes import collect_all_pods
pods = collect_all_pods()

In [22]:
a = pods['pod']['annotations']
s = a.str.contains("beta") & a.str.contains("hostname")
b = pods['pod'][s]
b.groupby(['namespace']).size().reset_index().set_index("namespace")

,0
namespace,
4tools,2
cc-dev,3
cc-perf2-qa-yancheng-dev,20
cc-wjx-local,1
cust-pro,1
default,3
demo,1
devops,1
druid-pe,9


In [23]:
import yaml
ignore_lists = [
    "creation_timestamp",
    "resource_version",
    "self_link",
    "uid",
    "claim_ref",
    "status",
    # "generation",
]
def clean_dict(d):
    if type(d) == list:
        for i in d:
            clean_dict(i)
        return
    if type(d) != dict:
        return
    for k, v in d.items():
        if (not v) or (k in ignore_lists):
            del d[k]
        else:
            clean_dict(d[k])
def yaml_out(d):
    clean_dict(d)
    return yaml.safe_dump(d, allow_unicode=True)

In [30]:
from k8sclient.K8SClient import K8SClient
k8sclient = K8SClient()
namespace = "monitor-essential-service"
dp = k8sclient.apiV1beta1.list_namespaced_replica_set(namespace).items

for d in dp:
    if d.metadata.owner_references:
        continue
    # print yaml_out(d.to_dict())

In [31]:
rs = k8sclient.apiV1beta1.list_namespaced_replica_set(namespace).items
a = rs[0].to_dict()
clean_dict(a)
# print yaml.safe_dump(a, allow_unicode=True)

In [32]:
a = dp[0].to_dict()
clean_dict(a)
# print yaml.safe_dump(a, allow_unicode=True)